In [ ]:
from PIL import Image, ImageFilter, ImageOps
import pyocr
import pyocr.builders
import pandas as pd
import glob
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

'''
便利な関数を宣言
'''
def cv2pil(image):
    new_image = image.copy()
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGRA2RGBA)
    new_image = Image.fromarray(new_image)
    return new_image
def pil2cv(image):
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image

def is_num(s):
    try:
        float(s)
    except ValueError:
        return False
    else:
        return True

In [ ]:
# pyocr.tesseract.TESSERACT_CMD = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [ ]:
cl_lst = []
fileName = []
engines = pyocr.get_available_tools()
engine = engines[0]

In [ ]:
engine

In [ ]:
'''
画像処理+文字認識
'''
cl_lst = []
fileName = []
engines = pyocr.get_available_tools()
engine = engines[0]
# files = glob.glob("./test/*")
files = glob.glob("./result_right/*")
files.sort()
for file in files:
    img = cv2.imread(file)
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.gray()
    im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_tr = im_gray[290:326, 620:800]

    # ret, im_otsu = cv2.threshold(img_tr, 140, 255, cv2.THRESH_OTSU)


    #膨張処理
    # kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    # dilate_img = cv2.dilate(pil2cv(im_otsu), kernel)
    #色反転
    im_rev = ImageOps.invert(cv2pil(im_gray))
    plt.imshow(im_rev)
    #文字認識
    txt = engine.image_to_string(im_rev, lang="jpn",
                                 builder=pyocr.builders.TextBuilder(tesseract_layout=11))
    at = txt.find("mg")
    print("==========", file, "==========")
    if at == -1:
        print("mg is not found")
        cl_lst.append(-1)
        fileName.append(file)
        continue
    cloline = txt[at-4:at]
    if is_num(cloline) == False:
        print("invalid number")
        print(cloline)
        cl_lst.append(-2)
        fileName.append(file)
        continue
    print(float(cloline))
    cl_lst.append(float(cloline))
    fileName.append(file)

In [ ]:
df = pd.DataFrame(cl_lst)

In [ ]:
df2 = pd.DataFrame(fileName)

In [ ]:
df3 = pd.concat([df2, df], axis=1)

In [ ]:
df3.to_excel('data.xlsx')